In [20]:
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

# modeling

from sklearn.metrics import mean_squared_error,r2_score, mean_absolute_error
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor,AdaBoostRegressor
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression,Ridge,Lasso
from sklearn.model_selection import RandomizedSearchCV, train_test_split
from xgboost import XGBRegressor
from sklearn.preprocessing import OneHotEncoder,StandardScaler
from sklearn.compose import ColumnTransformer

# import catboost
import warnings


In [3]:
data = pd.read_csv("data/raw.csv")
data.head()

,gender,race_ethnicity,parental_level_of_education,lunch,test_preparation_course,math_score,reading_score,writing_score
0,female,group B,bachelor's degree,standard,none,72,72,74
1,female,group C,some college,standard,completed,69,90,88
2,female,group B,master's degree,standard,none,90,95,93
3,male,group A,associate's degree,free/reduced,none,47,57,44
4,male,group C,some college,standard,none,76,78,75


In [8]:
x = data.drop("math_score", axis = 1)
y = data["math_score"]

In [10]:
x.info()

<class 'pandas.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 7 columns):
 #   Column                       Non-Null Count  Dtype
---  ------                       --------------  -----
 0   gender                       1000 non-null   str  
 1   race_ethnicity               1000 non-null   str  
 2   parental_level_of_education  1000 non-null   str  
 3   lunch                        1000 non-null   str  
 4   test_preparation_course      1000 non-null   str  
 5   reading_score                1000 non-null   int64
 6   writing_score                1000 non-null   int64
dtypes: int64(2), str(5)
memory usage: 54.8 KB


In [11]:
num_features = [feature for feature in x.columns if x[feature].dtype != "str"]
cat_features = [feature for feature in x.columns if x[feature].dtype == "str"]

print("num_features: ",num_features)
print("cat_features: ",cat_features)

num_features:  ['reading_score', 'writing_score']
cat_features:  ['gender', 'race_ethnicity', 'parental_level_of_education', 'lunch', 'test_preparation_course']


In [15]:
one_hot = OneHotEncoder()
scaler = StandardScaler()

preprocessor = ColumnTransformer([
    ("OneHotEncoder", one_hot, cat_features),
    ("StandardScaler", scaler, num_features)
])

In [17]:
processed_data = preprocessor.fit_transform(x)
processed_data

array([[ 1.        ,  0.        ,  0.        , ...,  1.        ,
         0.19399858,  0.39149181],
       [ 1.        ,  0.        ,  0.        , ...,  0.        ,
         1.42747598,  1.31326868],
       [ 1.        ,  0.        ,  0.        , ...,  1.        ,
         1.77010859,  1.64247471],
       ...,
       [ 1.        ,  0.        ,  0.        , ...,  0.        ,
         0.12547206, -0.20107904],
       [ 1.        ,  0.        ,  0.        , ...,  0.        ,
         0.60515772,  0.58901542],
       [ 1.        ,  0.        ,  0.        , ...,  1.        ,
         1.15336989,  1.18158627]], shape=(1000, 19))

In [18]:
processed_data.shape

(1000, 19)

In [21]:
x_train,x_test, y_train, y_test = train_test_split(processed_data,y,test_size=0.2,random_state=42)
x_train.shape, x_test.shape


((800, 19), (200, 19))

## Creating an evaluation function to give all metrics after model training

In [22]:
def evalutate_model(true_value,predicted):
    mae = mean_absolute_error(true_value, predicted)
    mse = mean_squared_error(true_value,predicted)
    rmse = np.sqrt(mean_squared_error(true_value,predicted))
    r2_square = r2_score(true_value,predicted)

    return mae,rmse, r2_square

## Modeling

In [23]:
models = {
    "Linear Regression": LinearRegression(),
    "Lasso": Lasso(),
    "Ridge" : Ridge(),
    "K-Neighbors Regressor": KNeighborsRegressor(),
    "Decision Tree" : DecisionTreeRegressor(),
    "Random Forest Refressor": RandomForestRegressor(),
    "XGB Regressor": XGBRegressor(),
    "AdaBoost Regressor": AdaBoostRegressor()
}

model_list = []
r2_list = []